In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix

# -----------------------------
# 配置：你的 Megalodon 模型路径
# -----------------------------
OBJ_PATH = "Megalodon.obj"   # 改成你的路径

def load_obj_vertices_faces(filename):
    """从 .obj 中读取 v 和 f，输出
       V: (N,3) 顶点坐标
       F: (M,3) 三角形顶点索引（0-based）
    """
    vertices = []
    faces = []
    with open(filename, "r") as f:
        for line in f:
            if line.startswith("v "):
                parts = line.strip().split()
                vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
            elif line.startswith("f "):
                parts = line.strip().split()[1:]
                # 支持  v   或  v/vt/vn  格式，只取第一个数字
                idx = [int(p.split("/")[0]) for p in parts]
                # 假定原始为三角形；如果是多边形可以做简单扇形三角剖分
                if len(idx) == 3:
                    faces.append(idx)
                elif len(idx) > 3:
                    # 扇形剖分: (0, i, i+1)
                    for i in range(1, len(idx) - 1):
                        faces.append([idx[0], idx[i], idx[i+1]])

    V = np.array(vertices, dtype=float)
    F = np.array(faces, dtype=int) - 1  # 转成 0-based
    return V, F

V_full, F_full = load_obj_vertices_faces(OBJ_PATH)

print("Full mesh:")
print("  Vertices:", V_full.shape)
print("  Faces   :", F_full.shape)

# 如果你已经有 CSV，可替换为：
# V_full = np.loadtxt("Megalodon_vertices.csv", delimiter=",", skiprows=1)
# F_full = np.loadtxt("Megalodon_faces.csv", delimiter=",", skiprows=1, dtype=int) - 1